In [149]:
import os

# Set env vars BEFORE importing huggingface modules
os.environ["HF_HOME"] = "/projects/sciences/computing/sheju347/.cache/huggingface"
os.environ["TRANSFORMERS_CACHE"] = "/projects/sciences/computing/sheju347/.cache/huggingface/hub"

# Now import huggingface modules
from huggingface_hub import constants

print("HF_HOME:", constants.HF_HOME)
print("HF_HUB_CACHE:", constants.HF_HUB_CACHE)

HF_HOME: /projects/sciences/computing/sheju347/.cache/huggingface
HF_HUB_CACHE: /projects/sciences/computing/sheju347/.cache/huggingface/hub


In [150]:

class TrainingData:
    def __init__(self, context: str, question: str, is_correct: bool):
        self.context = context
        self.question = question
        self.is_correct = is_correct


In [151]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# model_name = "google/flan-t5-large"  # or t5-base, t5-large
# model_name = "google-t5/t5-base"
# model_name = "google-t5/t5-small"
# model_name = "google-t5/t5-large"
model_name = "/projects/sciences/computing/sheju347/RAG/sentence_level_classifier/t5-large-epoch-10-5/checkpoint-9450"
# model_name = "/projects/sciences/computing/sheju347/RAG/sentence_level_classifier/t5-large-epoch-10-2/checkpoint-1363"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
print("done")

done


In [155]:
import re
import os

# QUESTION_COUNT = 10178

# log_file_list = [
#     "9-20-train_150_30_1st_H100.txt", 
#     "9-20-train_150_30_2nd_H100.txt", 
#     "9-21-train_150_30_3rd_H100.txt",
#     "9-23-train_150_30_4th_H100.txt",
#     "9-24-train_150_30_5th_H100.txt",
#                 ]
# log_file_list = ["10-7-150-30-3-sent_level_training_data.txt"]
# log_file_list = ["10-7-sent_level_test_data.txt"]
log_file_list = ["10-7-150-30-3-sent_level_test_data.txt"]
# log_file = "9-20-train_150_30_2nd_H100.txt"

# path = "../../notebooks/"
path = "./"

file_name_list = [path + log_file for log_file in log_file_list]

training_data_list = []

for file_name in file_name_list:
    with open(file_name) as f:
        is_reading_context = False
        context = ""
    
        is_reading_question = False
        question = ""
    
        for line in f:
            # if line == "\n":
            #     continue
    
            stripped_line = line.strip()
            if stripped_line == "Context:":
                is_reading_context = True
            elif stripped_line == "Question:":
                is_reading_context = False
                is_reading_question = True
            elif stripped_line.startswith("[output]"):
                is_correct = stripped_line[len("[output]"):] == "True"
                context = context.strip()
                question = question.strip()
                data = TrainingData(context, question, is_correct)
                # print("aaa", context, question, is_correct)
    
                training_data_list.append(data)
    
                # if len(training_data_list) % 100 == 0:
                #     print(f"finished processing {len(training_data_list)} data")
    
                # clear
                is_reading_context = False
                context = ""
                is_reading_question = False
                question = ""
                # break
            else:
                if is_reading_context:
                    context += line
                elif is_reading_question:
                    question += line

# training_data_list = training_data_list[:1000]
print(len(training_data_list))
sample = training_data_list[6]
print(sample.context)
print(sample.question)
print(training_data_list[6].is_correct)

1386
Twenty-five medical students were retained for longitudinal training and testing for four weeks.
A junior orthopaedic surgery resident is completing a carpal tunnel repair with the department chairman as the attending physician. During the case, the resident inadvertently cuts a flexor tendon. The tendon is repaired without complication. The attending tells the resident that the patient will do fine, and there is no need to report this minor complication that will not harm the patient, as he does not want to make the patient worry unnecessarily. He tells the resident to leave this complication out of the operative report. Which of the following is the correct next action for the resident to take?

[A] : Disclose the error to the patient and put it in the operative report
[B] : Tell the attending that he cannot fail to disclose this mistake
[C] : Report the physician to the ethics committee
[D] : Refuse to dictate the operative report
False


In [153]:
# Force set data to be balanced
adapted_data_list = []
incorrect_count = 0
for data in training_data_list:
    if data.is_correct == False:
        incorrect_count += 1

counter = 0
for data in training_data_list:
    if data.is_correct == False:
        adapted_data_list.append(data)
    else:
        if counter < incorrect_count + 0:
            adapted_data_list.append(data)
            counter += 1

training_data_list = adapted_data_list


# Change the label based on length
lengths = []
for data in training_data_list:
    lengths.append(len(data.context) + len(data.question))
import statistics
median_len = statistics.median(lengths)
print("median_len", median_len)

# for data in training_data_list:
#     if (len(data.context) + len(data.question)) > median_len:
#         data.is_correct = True
#     else:
#         data.is_correct = False
    

false_count = 0
true_count = 0
for data in training_data_list:
    if data.is_correct:
        true_count += 1
    else:
        false_count += 1
print(false_count, true_count)

median_len 909.5
439 439


In [156]:
# filename = "./data/cfimdb-train.txt"

# training_data_list = []

# with open(filename, 'r') as fp:
#     for line in fp:
#         label, org_sent = line.split(' ||| ')
#         # sent = org_sent.lower().strip()
#         # tokens = tokenizer.tokenize("[CLS] " + sent + " [SEP]")
#         label = int(label.strip())
#         # if label not in num_labels:
#         #     num_labels[label] = len(num_labels)
#         data = TrainingData(org_sent, "", label)
#         training_data_list.append(data)

# print(training_data_list[6].context, training_data_list[6].is_correct)

In [157]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Suppose training_data_list is your list of TrainingData objects
data = pd.DataFrame([{
    "context": d.context,
    "question": d.question,
    "label": int(d.is_correct)   # True->1, False->0
} for d in training_data_list])

# Train/val/test split (e.g., 70/15/15)
train_df, temp_df = train_test_split(data, test_size=0.3, random_state=42, stratify=data["label"])
val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42, stratify=temp_df["label"])

print(len(train_df), len(val_df), len(test_df))
# print(test_df.iloc[0]["context"])
# print(test_df.iloc[0]["question"])
print(test_df.iloc[1]["label"])

970 208 208
1


In [158]:
import torch
from torch.utils.data import Dataset

MAX_LEN = 1024 #512
TARGET_MAX_LEN = 2

class QADataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_len=MAX_LEN, target_max_len=TARGET_MAX_LEN):
        self.data = dataframe.reset_index(drop=True)
        self.tokenizer = tokenizer
        self.max_len = max_len
        self.target_max_len = target_max_len

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        context = self.data.loc[idx, "context"]
        question = self.data.loc[idx, "question"]
        label = self.data.iloc[idx]["label"]

        input_text = f"question: {question} context: {context}"
        target_text = "yes" if label == 1 else "no"

        # Keep return_tensors=None, let __getitem__ return 1D tensors
        encoding = self.tokenizer(
            input_text,
            truncation=True,
            padding="max_length",
            max_length=self.max_len,
            return_tensors=None
        )
        
        target_encoding = self.tokenizer(
            target_text,
            truncation=True,
            padding="max_length",
            max_length=self.target_max_len,
            return_tensors=None
        )

        # Convert to tensors (1D)
        input_ids = torch.tensor(encoding["input_ids"], dtype=torch.long)
        attention_mask = torch.tensor(encoding["attention_mask"], dtype=torch.long)
        labels = torch.tensor(target_encoding["input_ids"], dtype=torch.long)

        return {
            "input_ids": input_ids,
            "attention_mask": attention_mask,
            "labels": labels
        }

In [159]:
train_dataset = QADataset(train_df, tokenizer)
val_dataset = QADataset(val_df, tokenizer)
test_dataset = QADataset(test_df, tokenizer)
all_dataset = QADataset(data, tokenizer)

In [160]:
print(train_dataset[9]["input_ids"].tolist())
# print(len(train_dataset[2]["input_ids"].tolist()))
# print(train_dataset[1]["labels"])

[822, 10, 71, 2179, 24918, 7, 17, 19, 6908, 8, 3, 25120, 13, 3, 9, 3, 5771, 16291, 6035, 13, 8, 6722, 5, 621, 3, 9, 3117, 810, 13, 8, 6722, 11, 165, 280, 4005, 6, 3, 88, 4230, 7, 3, 9, 4516, 10, 3, 22917, 6, 3, 9, 2290, 2358, 19, 576, 18, 77, 89, 7633, 28, 204, 19601, 45, 8, 337, 6722, 384, 5, 8381, 8, 2290, 2358, 6, 975, 287, 155, 288, 999, 13, 796, 22213, 15107, 45, 321, 19601, 6986, 5, 3, 19494, 6, 8, 315, 22213, 15107, 45, 8, 19601, 33, 20828, 139, 3, 9, 775, 11, 3714, 6722, 24317, 5, 37, 6164, 5147, 6722, 24317, 19, 321, 5711, 11, 15109, 11, 19, 3, 9, 126, 6035, 45, 8, 6722, 384, 24, 2953, 8, 22494, 13, 7952, 5, 4073, 13, 8, 826, 19601, 19, 3919, 13, 3, 22725, 8, 756, 18, 297, 23, 782, 26, 433, 58, 784, 188, 908, 3, 10, 10395, 4008, 18, 14851, 52, 6722, 784, 279, 908, 3, 10, 3892, 17133, 221, 4638, 23, 4392, 6722, 784, 254, 908, 3, 10, 8704, 9, 18095, 784, 308, 908, 3, 10, 3, 31334, 23, 9, 6722, 2625, 10, 5433, 3, 476, 371, 553, 1003, 308, 1355, 77, 17856, 5673, 47, 6970, 16, 2242

In [161]:
sample = all_dataset[0]  # first element

print("Keys:", sample.keys())
print("Input IDs:", sample["input_ids"][:50])  # first 50 tokens
print("Attention Mask:", sample["attention_mask"][:50])
print("Labels:", sample["labels"])

# If you also want to see the decoded text:
print("\nDecoded input:")
print(tokenizer.decode(sample["input_ids"], skip_special_tokens=True))

print("\nDecoded label:")
print(tokenizer.decode(sample["labels"], skip_special_tokens=True))

Keys: dict_keys(['input_ids', 'attention_mask', 'labels'])
Input IDs: tensor([  822,    10,    71,  9212, 26429,   102,     9,    15,  4370,  3730,
         8141,    19,     3,  8828,     3,     9,   443,  6459, 11916,  2096,
           28,     8,  3066, 13404,    38,     8,  7078, 10027,     5,     3,
         2092,     8,   495,     6,     8,  8141,    16,     9,    26,  3027,
          295,   120,  8620,     3,     9,     3,  8902,   127,  2134,   106])
Attention Mask: tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1])
Labels: tensor([150,   1])

Decoded input:
question: A junior orthopaedic surgery resident is completing a carpal tunnel repair with the department chairman as the attending physician. During the case, the resident inadvertently cuts a flexor tendon. The tendon is repaired without complication. The attending tells the resident that the patient wi

In [162]:
from transformers import Trainer, TrainingArguments


output_dir = "/projects/sciences/computing/sheju347/RAG/sentence_level_classifier/t5-large-epoch-3" # "./results"

training_args = TrainingArguments(
    output_dir=output_dir,
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="./logs",
    load_best_model_at_end=True,
    metric_for_best_model="accuracy"
)

# def compute_metrics(eval_pred):
#     from sklearn.metrics import accuracy_score, f1_score
#     logits, labels = eval_pred
#     preds = logits.argmax(-1)
#     return {
#         "accuracy": accuracy_score(labels, preds),
#         "f1": f1_score(labels, preds)
#     }

def compute_metrics(eval_pred):
    from sklearn.metrics import accuracy_score, f1_score
    import torch

    # Extract logits and labels
    logits = eval_pred.predictions
    labels = eval_pred.label_ids

    # If logits is a tuple (common for seq2seq models), take the first element
    if isinstance(logits, tuple):
        logits = logits[0]

    # Convert logits to predicted token IDs
    pred_ids = torch.argmax(torch.tensor(logits), dim=-1)
    
    # Decode to text
    decoded_preds = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    
    # Map yes/no to 1/0
    y_pred = [1 if x.strip().lower() == "yes" else 0 for x in decoded_preds]
    
    # True labels
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    y_true = [1 if x.strip().lower() == "yes" else 0 for x in decoded_labels]

    return {
        "accuracy": accuracy_score(y_true, y_pred),
        "f1": f1_score(y_true, y_pred)
    }


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)


/tmp/ipykernel_91685/2230289905.py:60: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [163]:

# trainer.train()

In [164]:
from sklearn.metrics import accuracy_score

test_ds = all_dataset
# test_ds = Subset(all_dataset, range(800))
# test_ds = Subset(test_dataset, range(800))
# test_ds = val_dataset
# test_ds = test_dataset
# test_ds = Subset(train_dataset, range(1000, 3000))
# test_ds = train_dataset
print(len(test_ds))

preds = trainer.predict(test_ds)

print(preds.metrics)  # optional, shows overall metrics

# Get logits
logits = preds.predictions  # <--- use .predictions


# If logits is a tuple (common for seq2seq models), take the first element
if isinstance(logits, tuple):
    logits = logits[0]  # shape: (num_examples, seq_len, vocab_size)

print("logits", logits)
# Convert logits to token IDs
pred_ids = torch.argmax(torch.tensor(logits), dim=-1)

print ("pred_ids", pred_ids)
# Decode token IDs to text
decoded_preds = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
decoded_labels = tokenizer.batch_decode(preds.label_ids, skip_special_tokens=True)

print("decoded_preds", decoded_preds)
print("decoded_labels", decoded_labels)

# Convert yes/no to 1/0
y_pred = [1 if x.strip().lower() == "yes" else 0 for x in decoded_preds]
y_true = [1 if x.strip().lower() == "yes" else 0 for x in decoded_labels]

# Compute accuracy
accuracy = accuracy_score(y_true, y_pred)
print("Accuracy:", accuracy)


1386


{'test_loss': 2.477546215057373, 'test_model_preparation_time': 0.0056, 'test_accuracy': 0.4971139971139971, 'test_f1': 0.49820014398848095, 'test_runtime': 54.0057, 'test_samples_per_second': 25.664, 'test_steps_per_second': 12.832}
logits [[[-44.45292   -29.19883   -33.08519   ... -71.95722   -72.164246
   -72.082825 ]
  [-55.81641    -6.1869583 -32.18572   ... -74.494606  -74.89007
   -74.862236 ]]

 [[-43.602352  -28.392666  -32.448067  ... -71.04015   -71.25137
   -71.17189  ]
  [-55.26158    -6.458891  -32.295345  ... -74.12613   -74.537544
   -74.48744  ]]

 [[-43.4112    -28.15816   -32.26693   ... -70.53875   -70.7336
   -70.65748  ]
  [-55.649704   -6.377319  -32.353897  ... -74.53244   -74.934364
   -74.898575 ]]

 ...

 [[-37.81366   -23.851517  -28.087967  ... -64.35596   -64.59159
   -64.385826 ]
  [-57.868404   -6.9095516 -32.9035    ... -77.381485  -77.85263
   -77.77179  ]]

 [[-39.072563  -24.128502  -28.748554  ... -64.761314  -65.014145
   -64.81213  ]
  [-58.78525 

In [165]:
from sklearn.metrics import accuracy_score
import numpy as np
import torch
from torch.utils.data import Subset

# trainer.args.predict_with_generate = False  # make sure no .generate()

chunk_size = 500  # adjust depending on your GPU memory
all_logits = []
all_labels = []

for start in range(0, len(test_dataset), chunk_size):
    end = min(start + chunk_size, len(test_dataset))
    subset = Subset(test_dataset, range(start, end))
    preds = trainer.predict(subset)

    logits = preds.predictions
    if isinstance(logits, tuple):
        logits = logits[0]

    all_logits.append(logits)
    all_labels.append(preds.label_ids)

# Concatenate all chunks
logits = np.concatenate(all_logits, axis=0)
labels = np.concatenate(all_labels, axis=0)

print("logits shape:", logits.shape)

# Convert logits to predicted token IDs
pred_ids = torch.argmax(torch.tensor(logits), dim=-1)

# Only take the first token for yes/no classification
first_token_ids = pred_ids[:, 0]

# Decode predictions and labels
decoded_preds = tokenizer.batch_decode(first_token_ids, skip_special_tokens=True)
decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

print("decoded_preds sample:", decoded_preds[:20])
print("decoded_labels sample:", decoded_labels[:20])

# Map yes/no to 1/0
y_pred = [1 if x.strip().lower() == "yes" else 0 for x in decoded_preds]
y_true = [1 if x.strip().lower() == "yes" else 0 for x in decoded_labels]

# Compute accuracy
accuracy = accuracy_score(y_true, y_pred)
print("Accuracy:", accuracy)


KeyboardInterrupt: 

In [ ]:
preds = trainer.predict(test_dataset)
print(preds)
y_pred = preds.predictions.argmax(-1)
y_true = preds.label_ids

from sklearn.metrics import classification_report
print(classification_report(y_true, y_pred))

In [ ]:
y_preds = preds.predictions.argmax(-1)
count_predict_1 = 0
count_label_1 = 0
for i in range(len(test_dataset)):
    if i < 20:
        print(sum(test_dataset[i]["attention_mask"]), test_dataset[i]["labels"], y_preds[i])
    count_predict_1 += y_preds[i]
    count_label_1 += test_dataset[i]["labels"].item()

print(count_predict_1, "/", len(test_dataset), count_predict_1 / len(test_dataset))
print(count_label_1, "/", len(test_dataset), count_label_1 / len(test_dataset))